In [76]:
import pandas as pd
import psycopg2
from src import constants

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [65]:
data_connection = psycopg2.connect(database="gdelt_social_video", user="postgres")
data_cursor = data_connection.cursor()
features_connection = psycopg2.connect(database="video_article_retrieval", user="postgres")
features_cursor = features_connection.cursor()

# For all videos, sum up to probabilities of all labels
# Get all labels with their probability sum
# SELECT class, sum(probability) AS probability_sum FROM object_detection_yolo GROUP BY class ORDER BY probability_sum DESC


query = "SELECT platform, id, SUM(probability) as total, "
query += ",".join("SUM(CASE WHEN class='%s' THEN probability END) as %s" % (label, label.replace(" ", "_")) for label in constants.COCO_CLASS_NAMES)
query += " FROM object_detection_yolo GROUP BY (platform,id)"
features = pd.read_sql(query, con=features_connection)

In [66]:
features.head()

,platform,id,total,person,bicycle,car,motorbike,aeroplane,bus,train,...,toaster,sink,refrigerator,book,clock,vase,scissors,teddy_bear,hair_drier,toothbrush
0,facebook,100001732932719/1647247215343027,1.067463,1.067463,NaN,NaN,NaN,NaN,NaN,NaN,...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
1,facebook,100002259829821/1786627524755891,1002.008254,126.310780,NaN,813.577258,NaN,NaN,NaN,NaN,...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
2,facebook,100002259829821/1786745844744059,344.761922,171.186545,NaN,163.867663,NaN,NaN,NaN,NaN,...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
3,facebook,100004527049328/972499912910901,120.830634,69.977936,NaN,21.652411,NaN,NaN,NaN,NaN,...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
4,facebook,100004527049328/973448889482670,382.348741,93.056441,NaN,212.163793,0.517149,NaN,1.160108,NaN,...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None


In [19]:
len(features) # There are 2454 - 2261 = 193 videos where nothing was detected

2261

In [67]:
## Get the video durations and join it
# duration_query = "SELECT platform, id, duration FROM videos WHERE platform='facebook' AND crawling_status='Success'"
# durations =  pd.read_sql(duration_query, con=data_connection)
# features.join(durations, on=["platform", "id"])
# durations.head()

def get_duration(row):
    data_cursor.execute("SELECT duration FROM videos WHERE platform=%s AND id=%s", [row['platform'], row['id']])
    (duration,) = data_cursor.fetchone()
    return duration

features['duration'] = features.apply(get_duration, axis=1)
features.head()

,platform,id,total,person,bicycle,car,motorbike,aeroplane,bus,train,...,sink,refrigerator,book,clock,vase,scissors,teddy_bear,hair_drier,toothbrush,duration
0,facebook,100001732932719/1647247215343027,1.067463,1.067463,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,29510
1,facebook,100002259829821/1786627524755891,1002.008254,126.310780,NaN,813.577258,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,816264
2,facebook,100002259829821/1786745844744059,344.761922,171.186545,NaN,163.867663,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,55911
3,facebook,100004527049328/972499912910901,120.830634,69.977936,NaN,21.652411,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,177064
4,facebook,100004527049328/973448889482670,382.348741,93.056441,NaN,212.163793,0.517149,NaN,1.160108,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,138453


In [70]:
features = features.fillna(0)
features.head()

,platform,id,total,person,bicycle,car,motorbike,aeroplane,bus,train,...,sink,refrigerator,book,clock,vase,scissors,teddy_bear,hair_drier,toothbrush,duration
0,facebook,100001732932719/1647247215343027,1.067463,1.067463,0.0,0.000000,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,29510
1,facebook,100002259829821/1786627524755891,1002.008254,126.310780,0.0,813.577258,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,816264
2,facebook,100002259829821/1786745844744059,344.761922,171.186545,0.0,163.867663,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,55911
3,facebook,100004527049328/972499912910901,120.830634,69.977936,0.0,21.652411,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,177064
4,facebook,100004527049328/973448889482670,382.348741,93.056441,0.0,212.163793,0.517149,0.0,1.160108,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,138453


In [75]:
# Replace null with 0 and divide all sums by the duration
features = features.fillna(0)
for label in constants.COCO_CLASS_NAMES:
    column_name = label.replace(" ", "_")
    features[column_name] = features[column_name] / features["duration"]
    
features.head()

,platform,id,total,person,bicycle,car,motorbike,aeroplane,bus,train,...,sink,refrigerator,book,clock,vase,scissors,teddy_bear,hair_drier,toothbrush,duration
0,facebook,100001732932719/1647247215343027,1.067463,1.225786e-09,0.0,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29510
1,facebook,100002259829821/1786627524755891,1002.008254,1.895742e-10,0.0,1.221061e-09,0.000000e+00,0.0,0.000000e+00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,816264
2,facebook,100002259829821/1786745844744059,344.761922,5.476147e-08,0.0,5.242021e-08,0.000000e+00,0.0,0.000000e+00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,55911
3,facebook,100004527049328/972499912910901,120.830634,2.232033e-09,0.0,6.906304e-10,0.000000e+00,0.0,0.000000e+00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,177064
4,facebook,100004527049328/973448889482670,382.348741,4.854469e-09,0.0,1.106793e-08,2.697808e-11,0.0,6.051926e-11,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,138453


In [ ]:
# Next, the articles need to be preprocessed.